# How to complete this assignment
First, download [this Kaggle dataset](https://www.kaggle.com/hugomathien/soccer) and extract *sqlite* database. You may need to register at https://www.kaggle.com/ beforehand. Then complete 15 graded tasks below, the score is given in brackets. Finally submit the resulting `.ipynb` file to rs-app Auto-test.

- Do not delete or rename the variables given before the inscription `#your code here`, they are needed for the correct verification.
- Do not change the code in the last Notebook cell, it is required for the server check.
- Your Notebook must run completely without errors to be graded! Please check everything before submission by going *Cell -> Run All*

## Some important notes
- If you need to **calculate the number of "something"** that means we expect you to assign an Integer to the given variable
- If you need to **make a list of "something"** we expect you to assign a Python list with appropriate values to the given variable
- If you need to find a **specifiс player, day of the week, team, etc.** we expect you to assign a String with the full name of the entity to the given variable (`player_name`, day of week full name, `team_name`, etc.)

In [1]:
import sqlite3
import pandas as pd
import os

pd.set_option('display.max_column', None)

In [2]:
# Leave that code unchanged, it is required for the server check!
db = sqlite3.connect(os.environ.get("DB_PATH") or 'database.sqlite')

In [3]:
# You may load the data from SQL table directly to the Pandas dataframe as
player_data = pd.read_sql("SELECT * FROM Player;", db)

**Task 1 (0.25 point).** Calculate the number of players with a height between 180 and 190 inclusive

In [4]:
players_180_190 = len(player_data[(player_data["height"] <= 190) & (player_data["height"] >= 180)])

In [5]:
assert(isinstance(players_180_190, int))

**Task 2 (0.25 point).** Calculate the number of players born in 1980. <br>
**Hint:** you may want to cast your 'birthday' column to DateTime type by [pandas.to_datetime](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html)

In [6]:
player_data["birthday"] = pd.to_datetime(player_data["birthday"])
players_1980 = len(player_data[player_data["birthday"].dt.year == 1980])

In [7]:
assert(isinstance(players_1980, int))

**Task 3 (0.25 point).** Make a list of the top 10 players with the highest weight sorted in descending order. If there are several players with the same weight put them in the lexicographic order by name.

In [8]:
highest_players = player_data.sort_values(by=["weight", "player_name"], ascending=[False, True]).head(10)["player_name"].values.tolist()

In [9]:
assert(len(highest_players) == 10)
assert(isinstance(highest_players, list))
for i in range(10):
    assert(isinstance(highest_players[i], str))

**Task 4 (0.5 point).** Make a list of tuples containing years along with the number of players born in that year from 1980 up to 1990. <br>
**Structure example**: [(1980, 123), (1981, 140) ..., (1990, 83)] -> There were born 123 players in 1980, there were born 140 players in 1981 and etc.

In [10]:
players_amount_in_years = player_data[(player_data["birthday"].dt.year >= 1980) & (player_data["birthday"].dt.year <= 1990)]["birthday"] \
.dt.year.value_counts().sort_index()
years_born_players = list(zip(players_amount_in_years.index, players_amount_in_years))

In [11]:
assert(len(years_born_players) == 11)
assert(isinstance(years_born_players, list))
for i in range(10):
    assert(isinstance(years_born_players[i], tuple))
    assert(isinstance(years_born_players[i][0], int))
    assert(isinstance(years_born_players[i][1], int))

**Task 5 (0.5 point).** Calculate the mean and the standard deviation of the players' **height** with the name **Adriano**. <br>
**Note:** Name is represented by the first part of `player_name`.

In [12]:
adriano_name_players = player_data[player_data["player_name"].str.startswith("Adriano")]
adriano_mean, adriano_std = adriano_name_players["height"].mean(), adriano_name_players["height"].std()

In [13]:
assert(isinstance(adriano_mean, float))
assert(isinstance(adriano_std, float))

**Task 6 (0.75 point).** How many players were born on each day of the week? Find the day of the week with the minimum number of players born.

In [14]:
weekdays = {0 : "Monday", 1 : "Thuesday", 2 : "Wednesday", 3 : "Thursday", 4 : "Friday", 5 : "Saturday", 6 : "Sunday"}
dow_with_min_players_born = weekdays[player_data["birthday"].dt.weekday.value_counts().argmin()]

In [15]:
assert(isinstance(dow_with_min_players_born, str))

**Task 7 (0.75 point).** Find a league with the most matches in total. If there are several leagues with the same amount of matches, take the first in the lexical order.

In [16]:
matches_data = pd.read_sql("SELECT * FROM Match;", db)

In [17]:
leagues_data = pd.read_sql("SELECT * FROM League;", db)

In [18]:
matches_data = pd.merge(matches_data, leagues_data, how="left", on="country_id")

In [19]:
league_most_matches = matches_data["name"].value_counts().idxmax()

In [20]:
assert(isinstance(league_most_matches, str))

**Task 8 (1.25 point).** Find a player who participated in the largest number of matches during the whole match history. Assign a `player_name` to the given variable

In [21]:
players_attributes_data = pd.read_sql("SELECT * FROM Player_Attributes;", db)

In [34]:
player_names =  player_data["player_name"]
player_names.index = player_data["player_api_id"]
players_attributes_data = pd.merge(players_attributes_data, player_names, how="left", on="player_api_id")

In [23]:
subset_names = ["home_player_" + str(x) for x in range(1, 12)] + \
                ["away_player_" + str(x) for x in range(1, 12)]

In [24]:
dropped_nan_matches_data = matches_data.dropna(subset=subset_names)
matches_player_ids = dropped_nan_matches_data[subset_names].astype(int)
most_played_player_index = matches_player_ids.unstack().value_counts().idxmax()

In [25]:
max_matches_player = player_data[player_data["player_api_id"] == most_played_player_index].head()["player_name"].values[0]

In [26]:
assert(isinstance(max_matches_player, str))

**Task 9 (1.5 point).** List top-5 tuples of most correlated **player's characteristics** in the descending order of the absolute [Pearson's coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) value.

**Note 1:** Players characteristics are all the columns in `Player_Attributes` table except `[id, player_fifa_api_id, player_api_id, date, preferred_foot, attacking_work_rate, defensive_work_rate]`). <br>
**Note 2:** Exclude duplicated pairs from the list. E.g. ('gk_handling', 'gk_reflexes') and ('gk_reflexes', 'gk_handling') are duplicates, leave just one of them in the resulting list.

**Hint:** You may use [dataframe.corr()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html) for calculating pairwise Pearson correlation.

In [27]:
characteristics = players_attributes_data.columns.tolist()
characteristics = set(characteristics)
to_drop = set(["id", "player_fifa_api_id", "player_api_id", "date", "preferred_foot", "attacking_work_rate", "defensive_work_rate", "player_name"])
characteristics.difference_update(to_drop)
characteristics = list(characteristics)

In [87]:
correlations = players_attributes_data[characteristics].corr()

In [29]:
def get_pairs_to_drop(df):
    to_drop = set()
    columns = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            to_drop.add((columns[i], columns[j]))
    return to_drop

In [30]:
correlations = correlations.abs().unstack().drop(labels=get_pairs_to_drop(correlations))

In [31]:
top_correlated_features = correlations.sort_values(ascending=False).head().index.tolist()

In [32]:
assert(len(top_correlated_features) == 5)
assert(isinstance(top_correlated_features, list))
for i in range(5):
    assert(isinstance(top_correlated_features[i], tuple))
    assert(isinstance(top_correlated_features[i][0], str))
    assert(isinstance(top_correlated_features[i][1], str))

**Task 10 (2 points).** Find top-5 most similar players to **Neymar** whose names are given. The similarity is measured as [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance) between vectors of players' characteristics (described in the task above). Put their names in a vector in ascending order by Euclidean distance and sorted by `player_name` if the distance is the same<br>
**Note 1:** There are many records for some players in the `Player_Attributes` table. You need to take the freshest data (characteristics with the most recent `date`). <br>
**Note 2:** Use pure values of the characteristics even if you are aware of such preprocessing technics as normalization. <br>
**Note 3:** Please avoid using any built-in methods for calculating the Euclidean distance between vectors, think about implementing your own.

In [164]:
neymar_data = players_attributes_data[players_attributes_data["player_name"] == "Neymar"]
neymar_latest_data = neymar_data.sort_values(by="date", ascending=False)[characteristics].iloc[0]
neymar_latest_data

heading_accuracy      62.0
free_kick_accuracy    79.0
aggression            56.0
crossing              72.0
jumping               61.0
strength              45.0
gk_handling            9.0
stamina               79.0
reactions             86.0
gk_positioning        15.0
balance               84.0
curve                 78.0
ball_control          93.0
dribbling             94.0
agility               92.0
long_shots            73.0
marking               21.0
sprint_speed          90.0
gk_diving              9.0
sliding_tackle        33.0
gk_reflexes           11.0
interceptions         36.0
finishing             88.0
shot_power            78.0
gk_kicking            15.0
potential             94.0
standing_tackle       24.0
long_passing          74.0
vision                79.0
positioning           89.0
short_passing         78.0
acceleration          91.0
penalties             81.0
volleys               83.0
overall_rating        90.0
Name: 131464, dtype: float64

In [200]:
import numpy as np
players_attributes_data["similarity_to_Neymar"] = players_attributes_data[characteristics].sub(neymar_latest_data, axis="columns") \
.apply(lambda x: x**2).sum(axis="columns").apply(np.sqrt)
players_attributes_data = players_attributes_data.dropna(subset=characteristics)

In [204]:
player_names = players_attributes_data["player_name"].unique()
players_attributes_data = players_attributes_data.sort_values(by="date", ascending=False)
latest_players_attributes_data = players_attributes_data.drop_duplicates(subset=["player_name"]) \
.sort_values(by="similarity_to_Neymar", ascending=True)
player_names = latest_players_attributes_data["player_name"]
latest_players_attributes_data.head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,player_name,similarity_to_Neymar
131464,131465,190871,19533,2016-02-04 00:00:00,90.0,94.0,right,high,medium,72.0,88.0,62.0,78.0,83.0,94.0,78.0,79.0,74.0,93.0,91.0,90.0,92.0,86.0,84.0,78.0,61.0,79.0,45.0,73.0,56.0,36.0,89.0,79.0,81.0,21.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,Neymar,0.000000
140773,140774,211110,325916,2016-05-05 00:00:00,81.0,87.0,left,medium,medium,72.0,87.0,60.0,78.0,80.0,89.0,81.0,82.0,58.0,90.0,90.0,85.0,91.0,82.0,85.0,77.0,69.0,79.0,55.0,83.0,48.0,31.0,85.0,79.0,76.0,15.0,21.0,21.0,9.0,11.0,9.0,10.0,11.0,Paulo Dybala,33.837849
47247,47248,183277,107417,2016-01-28 00:00:00,88.0,90.0,right,high,medium,80.0,81.0,57.0,85.0,79.0,94.0,82.0,79.0,82.0,90.0,93.0,87.0,92.0,86.0,90.0,79.0,59.0,74.0,63.0,82.0,54.0,41.0,84.0,86.0,86.0,25.0,27.0,22.0,11.0,12.0,6.0,8.0,8.0,Eden Hazard,34.957117
64607,64608,170369,41157,2016-01-07 00:00:00,79.0,80.0,left,high,low,74.0,81.0,61.0,73.0,79.0,85.0,73.0,80.0,71.0,85.0,87.0,86.0,89.0,80.0,89.0,80.0,65.0,68.0,45.0,79.0,53.0,28.0,80.0,70.0,70.0,21.0,28.0,23.0,6.0,9.0,11.0,7.0,5.0,Giovani dos Santos,38.314488
16460,16461,9014,30834,2016-01-28 00:00:00,89.0,89.0,left,high,low,80.0,85.0,51.0,85.0,86.0,93.0,86.0,83.0,73.0,90.0,90.0,90.0,91.0,91.0,91.0,86.0,61.0,73.0,65.0,90.0,47.0,39.0,88.0,84.0,80.0,29.0,26.0,26.0,10.0,8.0,11.0,5.0,15.0,Arjen Robben,40.024992


In [205]:
neymar_similarities = player_names[1:6].tolist()
neymar_similarities

['Paulo Dybala',
 'Eden Hazard',
 'Giovani dos Santos',
 'Arjen Robben',
 'Lionel Messi']

In [188]:
assert(len(neymar_similarities) == 5)
assert(isinstance(neymar_similarities, list))
for i in range(5):
    assert(isinstance(neymar_similarities[i], str))

**Task 11 (1 point).** Calculate the number of home matches played by the **Borussia Dortmund** team in **Germany 1. Bundesliga** in season **2008/2009**

In [42]:
teams_data = pd.read_sql("SELECT * FROM Team;", db)

In [43]:
team_names =  teams_data["team_long_name"]
team_names.index = teams_data["team_api_id"]

In [44]:
matches_data = pd.merge(matches_data, team_names, how="left", 
                        left_on="home_team_api_id", right_on="team_api_id")
matches_data = pd.merge(matches_data, team_names, how="left", 
                        left_on="away_team_api_id", right_on="team_api_id", suffixes=["_home", "_away"])

In [45]:
borussia_bundesliga_2008_2009_matches = len(matches_data[(matches_data["season"] == "2008/2009") 
                                                         & (matches_data["name"] == "Germany 1. Bundesliga")
                                                        & (matches_data["team_long_name_home"] == "Borussia Dortmund")])

In [46]:
assert(isinstance(borussia_bundesliga_2008_2009_matches, int))

**Task 12 (1 point).** Find a team having the most matches (both home and away!) in the **Germany 1. Bundesliga** in **2008/2009** season. Return number of matches.

In [47]:
home_matches_teams = matches_data[(matches_data["season"] == "2008/2009") \
                & (matches_data["name"] == "Germany 1. Bundesliga")]["team_long_name_home"].value_counts()
away_matches_teams = matches_data[(matches_data["season"] == "2008/2009") \
                & (matches_data["name"] == "Germany 1. Bundesliga")]["team_long_name_away"].value_counts()
total_matches_teams = home_matches_teams + away_matches_teams

In [48]:
team_most_matches_bundesliga_2008_2009 = int(total_matches_teams.max())

In [49]:
assert(isinstance(team_most_matches_bundesliga_2008_2009, int))

**Task 13 (1 point).** Count total number of **Arsenal** matches (both home and away!) in the **2015/2016** season which they have won. <br><br>
**Note:** Winning a game means scoring **more** goals than an opponent.

In [50]:
arsenal_won_matches_2015_2016 = int(len(matches_data[(matches_data["season"] == "2015/2016") 
                                                 & (matches_data["team_long_name_home"] == "Arsenal")
                                                 & (matches_data["home_team_goal"] > matches_data["away_team_goal"])]) + \
                                len(matches_data[(matches_data["season"] == "2015/2016") 
                                                 & (matches_data["team_long_name_away"] == "Arsenal")
                                                 & (matches_data["home_team_goal"] < matches_data["away_team_goal"])]))

In [51]:
assert(isinstance(arsenal_won_matches_2015_2016, int))

**Task 14 (2 points).** Find a team with the highest win rate in the **2015/2016** season. Win rate means won matches / all matches. If there are several teams with the highest win rate return the first by name in lexical order

In [95]:
season_matches_data = matches_data[matches_data["season"] == "2015/2016"]
team_names_home = season_matches_data["team_long_name_home"].unique()
team_names_away = season_matches_data["team_long_name_away"].unique()
assert(team_names_home.sort() == team_names_away.sort())
#They are the same

In [105]:
teams_winrate = pd.Series(index=team_names_home)
season_matches_data["won_team"] = "Draw"
home_won = season_matches_data["home_team_goal"] > season_matches_data["away_team_goal"]
away_won = season_matches_data["home_team_goal"] < season_matches_data["away_team_goal"]
season_matches_data.loc[home_won, "won_team"] = season_matches_data[home_won]["team_long_name_home"]
season_matches_data.loc[away_won, "won_team"] = season_matches_data[away_won]["team_long_name_away"]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._

In [106]:
season_matches_data["won_team"].value_counts()

Draw                   855
Paris Saint-Germain     30
FC Barcelona            29
SL Benfica              29
Juventus                29
                      ... 
Hellas Verona            5
Polonia Bytom            4
SC Cambuur               3
ES Troyes AC             3
Aston Villa              3
Name: won_team, Length: 189, dtype: int64

In [108]:
season_matches_team_winrates = pd.Series(index=team_names_home)
for team in team_names_home:
  season_matches_team_winrates[team] = len(season_matches_data[season_matches_data["won_team"] == team]) / \
                                        len(season_matches_data[(season_matches_data["team_long_name_home"] == team) | 
                                                                (season_matches_data["team_long_name_away"] == team)])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


'SL Benfica'

In [110]:
team_highest_winrate_2015_2016 = season_matches_team_winrates.idxmax()

In [111]:
assert(isinstance(team_highest_winrate_2015_2016, str))

**Task 15 (2 points).** Determine the team with the maximum days' gap between matches in **England Premier League 2010/2011 season**. Return number of days in that gap. <br>
**Note**: a *gap* means the number of days between two consecutive matches of the same team.

In [118]:
matches_data_epl = matches_data[(matches_data["season"] == "2010/2011") & (matches_data["name"] == "England Premier League")]

In [119]:
matches_data_epl.loc[:, "date"] = pd.to_datetime(matches_data_epl["date"])

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [123]:
matches_data_epl = matches_data_epl.sort_values(by="date")
matches_data_epl.head()

,id_x,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA,id_y,name,team_long_name_home,team_long_name_away
2488,2489,1729,1729,2010/2011,1,2010-08-14,839796,10252,8654,3,0,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,5.0,5.0,1.0,2.0,4.0,6.0,8.0,1.0,3.0,5.0,7.0,9.0,5.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,9.0,11.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,7.0,11.0,30380.0,30357.0,161414.0,24211.0,24136.0,139671.0,30892.0,38609.0,38807.0,23354.0,26165.0,36374.0,35110.0,109897.0,23818.0,26348.0,36394.0,24223.0,37169.0,34590.0,30734.0,34543.0,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><corners>1</corners></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>67</comment><event...,2.00,3.30,4.0,1.85,3.40,4.10,1.80,3.4,4.0,1.83,3.4,4.33,NaN,NaN,NaN,1.91,3.25,4.33,2.00,3.4,3.75,2.00,3.25,4.20,2.00,3.25,3.75,1.80,3.40,4.33,1729,England Premier League,Aston Villa,West Ham United
2489,2490,1729,1729,2010/2011,1,2010-08-14,839797,8655,8668,1,0,1.0,2.0,4.0,6.0,8.0,1.0,3.0,5.0,7.0,9.0,5.0,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,5.0,5.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,7.0,11.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,9.0,11.0,30622.0,30658.0,19020.0,23921.0,30739.0,23927.0,186137.0,30342.0,23916.0,25667.0,21613.0,31465.0,30371.0,23268.0,24216.0,24846.0,24006.0,39618.0,24011.0,36012.0,30876.0,30735.0,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>45</comment><event...,2.88,3.25,2.5,2.75,3.25,2.45,2.55,3.2,2.5,2.80,3.3,2.50,NaN,NaN,NaN,2.88,3.20,2.50,2.75,3.3,2.50,3.10,3.30,2.38,2.90,3.20,2.40,2.70,3.25,2.50,1729,England Premier League,Blackburn Rovers,Everton
2490,2491,1729,1729,2010/2011,1,2010-08-14,839799,8559,9879,0,0,1.0,2.0,4.0,6.0,8.0,2.0,4.0,8.0,6.0,4.0,6.0,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,5.0,5.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,10.0,10.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,9.0,11.0,23932.0,26454.0,23783.0,40128.0,24728.0,130670.0,35532.0,33633.0,24455.0,34261.0,23934.0,35477.0,37266.0,23780.0,26777.0,33045.0,24020.0,25253.0,30338.0,34574.0,24737.0,24741.0,<goal />,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...

In [125]:
home_teams = matches_data_epl["team_long_name_home"].unique()
away_teams = matches_data_epl["team_long_name_away"].unique()
assert(home_teams.sort() == away_teams.sort())

In [133]:
match_dates_for_teams = {}
for team in home_teams:
  match_dates_for_teams[team] = matches_data_epl[(matches_data_epl["team_long_name_home"] == team) | 
                                                 (matches_data_epl["team_long_name_away"] == team)]["date"]
#match_dates_for_teams

In [145]:
team_differences = {}
for team, dates in match_dates_for_teams.items():
  team_differences[team] = dates.diff()

team_max_differences = {}
for team, diffs in team_differences.items():
  team_max_differences[team] = diffs.max()


18

In [146]:
highest_gap_england_2010_2011 = max(list(team_max_differences.items()), key=lambda x: x[1])[1].days

In [147]:
assert(isinstance(highest_gap_england_2010_2011, int))

### Warning! Do not change anything in the area below

In [ ]:
with open('student_answers.txt', 'w') as file:
    file.write(f"{players_180_190}\n")
    file.write(f"{players_1980}\n")
    file.write(f"{highest_players}\n")
    file.write(f"{years_born_players}\n")
    file.write(f"{round(adriano_mean, 3)} {round(adriano_std, 3)}\n")
    file.write(f"{dow_with_min_players_born}\n")
    file.write(f"{league_most_matches}\n")
    file.write(f"{max_matches_player}\n")
    file.write(f"{';'.join(['%s,%s' % tup for tup in top_correlated_features])};\n")
    file.write(f"{neymar_similarities}\n")
    file.write(f"{borussia_bundesliga_2008_2009_matches}\n")
    file.write(f"{team_most_matches_bundesliga_2008_2009}\n")
    file.write(f"{arsenal_won_matches_2015_2016}\n")
    file.write(f"{team_highest_winrate_2015_2016}\n")
    file.write(f"{highest_gap_england_2010_2011}\n")